In [1]:
# 假设对函数 y=2x的转置x，关于列向量x求导
import torch

In [2]:
x = torch.arange(4.0)

In [3]:
x

tensor([0., 1., 2., 3.])

In [4]:
# 在计算 y 关于 x 的梯度之前，需要一个地方来存储梯度
x.requires_grad_(True) # 等价于 x = torch.arange(4.0, requires_grad=True)

tensor([0., 1., 2., 3.], requires_grad=True)

In [5]:
x.grad

In [8]:
y = 2 * torch.dot(x,x) # 隐式构造计算图

In [9]:
y

tensor(28., grad_fn=<MulBackward0>)

In [11]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()

In [12]:
x.grad

tensor([ 0.,  4.,  8., 12.])

In [14]:
x.grad == 4 * x

tensor([True, True, True, True])

In [15]:
# 现在计算x的另一个函数
# 在默认情况下，pytorch会累积梯度，需要清除之前的值
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [16]:
y = x.sum()

In [17]:
y.backward()

In [18]:
x.grad

tensor([1., 1., 1., 1.])

In [21]:
# 深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和
# 对非标量调用‘backward’需要传入一个‘gradient’参数，该参数指定微分函数
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [22]:
y = x * x

In [23]:
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [24]:
y.sum()

tensor(14., grad_fn=<SumBackward0>)

In [25]:
y.sum().backward()

In [27]:
x.grad

tensor([0., 2., 4., 6.])

In [28]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [58]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [59]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [60]:
y = x * x

In [61]:
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [62]:
u = y.detach() # 把y当做一个常数，不是关于x的函数

In [63]:
u # u就是一个常熟，它的值就是x*x的值
  # u此时是一个常数张量，不参与梯度的计算，所以最后 z=u*x 的求导就是 u

tensor([0., 1., 4., 9.])

In [64]:
z = u * x

In [65]:
z

tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>)

In [66]:
z.sum()

tensor(36., grad_fn=<SumBackward0>)

In [67]:
z.sum().backward()

In [68]:
x.grad == u

tensor([True, True, True, True])

In [69]:
x.grad

tensor([0., 1., 4., 9.])

In [70]:
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [71]:
y.sum().backward()

In [72]:
x.grad == 2 * x

tensor([True, True, True, True])

In [74]:
# 即使构建函数的计算图需要通过Python控制流（例如：条件、循环或任意函数调用），仍然可以计算得到的变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [75]:
a = torch.randn(size=(), requires_grad=True)

In [76]:
d = f(a)

In [77]:
d.backward()

In [78]:
a.grad

tensor(3276800.)

In [79]:
a.grad == d / a

tensor(True)